In [1]:
# import Pkg; Pkg.add("CorpusLoaders")

In [1]:
using Knet, Random, Plots, IterTools
using Statistics: mean
using Base.Iterators: flatten
# using Embeddings

In [2]:
S = 40 # max sentence length
B = 50 # batch size
D = 100 # embedding size
C = 6 # number of classes
Ci = 1 # number of input channels
Ks = [3,4,5] # kernel_sizes
Co = 100 # output channels per kernel size
dp = 0.5; # dropout probability

In [3]:
w2i = Dict{String,Int}()
yw2i = Dict{String,Int}()
unk = get!(w2i, "<unk>", 1+length(w2i))
pad = get!(w2i, "<pad>", 1+length(w2i))

2

In [4]:
# trnlines = readlines("./TREC/train_5500.label.txt")
# ytrn = [split(i)[1] for i in trnlines]
# ytrn = [split(i, ":")[1] for i in ytrn]
# xtrn = [split(i)[2:end] for i in trnlines];

In [5]:
# l = length.(xtrn)

# using StatsBase;
# a = countmap(l);

# b = hcat([[key, val] for (key, val) in a]...)';

# sort(collect(a), by=x->x[2]);

In [6]:
# tstlines = readlines("./TREC/TREC_10.label.txt")
# ytst = [split(i)[1] for i in tstlines]
# ytst = [split(i, ":")[1] for i in ytst]
# xtst = [split(i)[2:end] for i in tstlines]

In [7]:
function readdata(trnf, tstf, S)
    ixtrn = []
    ixtst = []
    
    # read lines
    trnlines = readlines(trnf)
    
    # split label from sentence 
    ytrn = [split(i)[1] for i in trnlines]
    ytrn = [split(i, ":")[1] for i in ytrn]
    xtrn = [split(i)[2:end] for i in trnlines]
    
    # encode label
    ytrn = [get!(yw2i, w, 1+length(yw2i)) for w in ytrn]
    
    
    tstlines = readlines(tstf)
    ytst = [split(i)[1] for i in tstlines]
    ytst = [split(i, ":")[1] for i in ytst]
    ytst = [get!(yw2i, w, 1+length(yw2i)) for w in ytst]
    xtst = [split(i)[2:end] for i in tstlines]
    
    # encode sentences and pad or truncate
    for (i, line) in enumerate(xtrn)
        words = [get!(w2i, w, 1+length(w2i)) for w in line]
        # pad or truncate
        if length(words) >= S
            words = words[1:S]
        else
            words = [words; repeat([pad], S-length(words))]
        end
        push!(ixtrn, words)
    end
    
    for (i, line) in enumerate(xtst)
        words = [get(w2i, w, unk) for w in line]
        # pad or truncate
        if length(words) >= S
            words = words[1:S]
        else
            words = [words; repeat([pad], S-length(words))]
        end
        push!(ixtst, words)
    end
    
    # concatenate to one big matrix
    ixtrn, ixtst = hcat(ixtrn...), hcat(ixtst...)
    
    # create iterators
    dtrn, dtst = minibatch(ixtrn, ytrn, B, shuffle=true), minibatch(ixtst, ytst, B, shuffle=true)
    global V = length(w2i) # vocab size
    return dtrn, dtst
end

readdata (generic function with 1 method)

In [8]:
dtrn, dtst = readdata("./TREC/train_5500.label.txt", "./TREC/TREC_10.label.txt", S)

(Knet.Data{Tuple{Array{Int64,2},Array{Int64,1}}}([3 13 … 13 13; 4 14 … 37 9450; … ; 2 2 … 2 2; 2 2 … 2 2], [1 2 … 5 2], 50, 5452, false, 5403, 1:5452, true, (40, 5452), (5452,), Array{Int64,2}, Array{Int64,1}), Knet.Data{Tuple{Array{Int64,2},Array{Int64,1}}}([3 13 … 13 13; 2224 2319 … 37 37; … ; 2 2 … 2 2; 2 2 … 2 2], [5 6 … 2 1], 50, 500, false, 451, 1:500, true, (40, 500), (500,), Array{Int64,2}, Array{Int64,1}))

In [9]:
# Embed layer # may load pretrained embeddings instead
struct Embed; w; end
Embed(vocabsize::Int, embedsize::Int) = Embed(param(embedsize, vocabsize))
(l::Embed)(x) = reshape(l.w[:,x], (size(l.w,1),size(x,1),1,size(x,2)) ) # E,S,Cx,B

In [10]:
struct Dense; w; b; f; p; end
(d::Dense)(x) = d.f.(d.w * mat(dropout(x,d.p)) .+ d.b) # mat reshapes 4-D tensor to 2-D matrix so we can use matmul
Dense(i::Int,o::Int,f=relu;pdrop=0) = Dense(param(o,i), param0(o), f, pdrop)

Dense

In [11]:
struct Conv; w; b; f; p; end
function (c::Conv)(x)
    conved = conv4(c.w, dropout(x,c.p)) .+ c.b
    return c.f.(pool(conved; window=(size(conved, 1), size(conved, 2))))
end
Conv(w1::Int,w2::Int,cx::Int,cy::Int,f=relu;pdrop=0) = Conv(param(w1,w2,cx,cy), param0(1,1,cy,1), f, pdrop)

Conv

In [14]:
struct cnntext
    V # vocab size
    D # embedding size
    C # number of classes
    Ci # one
    Ks # kernel_sizes
    Co # number of each of the sizes
    dp # dropout probability
    embed::Embed # embedding layer
    fc::Dense 
    conv1::Conv
    conv2::Conv
    conv3::Conv
end

function (c::cnntext)(x)
    x = c.embed(x)  # E, S, Cx (1), B
    x1, x2, x3 = c.conv1(x), c.conv2(x), c.conv3(x) # 
    x = cat(x1,x2,x3, dims=1)
    x = c.fc(x)
end

(c::cnntext)(x,y) = nll(c(x),y)

(c::cnntext)(d::Knet.Data) = mean(c(x,y) for (x,y) in d)

function cnntext(V, D, C, Ci, Ks, Co, dp)
    cnntext(V, D, C, Ci, Ks, Co, dp, Embed(V,D), Dense(length(Ks)*Co,C,identity,pdrop=dp), Conv(D,Ks[1],Ci,Co), Conv(D,Ks[3],Ci,Co), Conv(D,Ks[3],Ci,Co))
end

cnntext

In [15]:
model = cnntext(V, D, C, Ci, Ks, Co, dp)

cnntext(9450, 100, 6, 1, [3, 4, 5], 100, 0.5, Embed(P(KnetArray{Float32,2}(100,9450))), Dense(P(KnetArray{Float32,2}(6,300)), P(KnetArray{Float32,1}(6)), identity, 0.5), Conv(P(KnetArray{Float32,4}(100,3,1,100)), P(KnetArray{Float32,4}(1,1,100,1)), NNlib.relu, 0), Conv(P(KnetArray{Float32,4}(100,5,1,100)), P(KnetArray{Float32,4}(1,1,100,1)), NNlib.relu, 0), Conv(P(KnetArray{Float32,4}(100,5,1,100)), P(KnetArray{Float32,4}(1,1,100,1)), NNlib.relu, 0))

In [ ]:
model(first(dtrn)[1])

In [24]:
nepochs = 2

2

In [25]:
function trainresults(file,model; o...)
    if (print("Train from scratch? "); readline()[1]=='y')
        r = ((model(dtrn), model(dtst), zeroone(model,dtrn), zeroone(model,dtst))
             for x in takenth(progress(adam(model,ncycle(dtrn,nepochs))),length(dtrn)))
        r = reshape(collect(Float32,flatten(r)),(4,:))
        Knet.save(file,"results",r)
        Knet.gc() # To save gpu memory
    else
        isfile(file) || return
        r = Knet.load(file,"results")
    end
    println(minimum(r,dims=2))
    return r
end

trainresults (generic function with 1 method)

In [26]:
cnn = trainresults("cnntextTREC.jld2", model);

Train from scratch? stdin> y


┣███████████████████▉┫ [100.00%, 218/218, 00:15/00:15, 14.67i/s] 


Float32[0.1734474; 0.3351663; 0.037247706; 0.12]


In [27]:
cnn

4×2 Array{Float32,2}:
 0.602168  0.173447 
 0.677961  0.335166 
 0.195046  0.0372477
 0.222     0.12     

In [45]:
# const w2v = load_embeddings(Word2Vec)

In [46]:
# const get_word_index = Dict(word=>ii for (ii,word) in enumerate(w2v.vocab))

# function get_embedding(word)
#     ind = get_word_index[word]
#     emb = w2v.embeddings[:,ind]
#     return emb
# end

In [47]:
# get_embedding("blue")

In [9]:
# This program has requested access to the data dependency word2vec 300d.
# which is not currently installed. It can be installed automatically, and you will not see this message again.

# Pretrained Word2Vec Word emeddings
# Website: https://code.google.com/archive/p/word2vec/
# Author: Mikolov et al.
# Year: 2013

# Pre-trained vectors trained on part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases.

# Paper:
#     Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, and Jeffrey Dean. Distributed Representations of Words and Phrases and their Compositionality. In Proceedings of NIPS, 2013.



# Do you want to download the dataset from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz to "/home/saif/.julia/datadeps/word2vec 300d"?
# [y/n]
# stdin> y
# const w2v = load_embeddings(Word2Vec)

This program has requested access to the data dependency word2vec 300d.
which is not currently installed. It can be installed automatically, and you will not see this message again.

Pretrained Word2Vec Word emeddings
Website: https://code.google.com/archive/p/word2vec/
Author: Mikolov et al.
Year: 2013

Pre-trained vectors trained on part of Google News dataset (about 100 billion words). The model contains 300-dimensional vectors for 3 million words and phrases.

Paper:
    Tomas Mikolov, Ilya Sutskever, Kai Chen, Greg Corrado, and Jeffrey Dean. Distributed Representations of Words and Phrases and their Compositionality. In Proceedings of NIPS, 2013.



Do you want to download the dataset from https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz to "/home/saif/.julia/datadeps/word2vec 300d"?
[y/n]
stdin> y


Embeddings.EmbeddingTable{Array{Float32,2},Array{String,1}}(Float32[0.06731994 0.05295623 … -0.21142985 0.013637338; -0.05344657 0.06545979 … -0.0087888 -0.07428761; … ; -0.0073346887 0.010894641 … -0.0040515745 0.015611163; -0.0051456536 -0.047072206 … -0.034157887 0.039655942], ["</s>", "in", "for", "that", "is", "on", "##", "The", "with", "said"  …  "#-###-PA-PARKS", "Lackmeyer", "PERVEZ", "KUNDI", "Budhadeb", "Nautsch", "Antuane", "tricorne", "VISIONPAD", "RAFFAELE"])